# Evaluation

In [1]:
import json
import os

import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt

## Summary Length

#### Run Evaluation

In [ ]:
# if not os.path.exists("./evaluation/evaluatedSummarys.csv"):
#     evalDF = pd.DataFrame(columns=["jira", "ticketId", "evolution", "outputUri", "violation_actual", "violation_predicted", "summary_old", "length_old", "summary_new", "length_new", "correction_in_range"])
# else:
#     evalDF = pd.read_csv("./evaluation/evaluatedSummarys.csv")

In [ ]:
directory = "./evaluation/summary/"

evalDF = pd.DataFrame(columns=["jira", "ticketId", "evolution", "output_uri", "violation_actual", "violation_predicted", "summary_old", "length_old", "summary_new", "length_new", "correction_in_range"])

actual = np.array([])
predicted = np.array([])

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    outputUri = directory + filename

    with open(outputUri) as f:
        result = json.load(f)
    
    actual = np.append(actual, result["violation_actual"])
    predicted = np.append(predicted, result["output"]["violation_predicted"])

    if (len(result["output"]["summary_new"]) <= 70) & (len(result["output"]["summary_new"]) >= 39):
        in_range = "TRUE"
    else:
        in_range = "FALSE"

    new_row = {
        'jira': result["input_data"]["jira"], 
        'ticketId': result["input_data"]["id"], 
        'evolution': result["input_data"]["evolution"], 
        'output_uri': outputUri, 
        'violation_actual': result["violation_actual"], 
        'violation_predicted': result["output"]["violation_predicted"],
        'summary_old': result["output"]["summary_old"],
        'length_old': len(result["output"]["summary_old"]), 
        'summary_new': result["output"]["summary_new"],
        'length_new': len(result["output"]["summary_new"]), 
        'correction_in_range': in_range
        }

    evalDF.loc[len(evalDF)]=new_row

# Save data to csv
evalDF.to_csv("./evaluation/evaluatedSummarys.csv", index=False)

In [ ]:
confusion_matrix = metrics.confusion_matrix(actual, predicted, labels=["TRUE", "FALSE"])
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["TRUE", "FALSE"])

In [ ]:
cm_display.plot()
plt.show()

In [ ]:
Accuracy = metrics.accuracy_score(actual, predicted)
Accuracy